利用Black-Scholes Pricing Formula以及真實選擇權價格，透過Bisection Search反求近似的隱含波動度。

In [1]:
from numpy import exp
from numpy.lib.scimath import log, sqrt
from scipy.stats import norm

In [2]:
# Black-Scholes formula for european call price in terms of theory.
def Black_Scholes_Call(S, K, r, T, sigma):
    d1 = (log(S/K) + (r + 0.5 * sigma**2) * T)/(sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    return S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)

# Then use put-call parity, we can also find european put price in terms of theory.
def European_Put(S, K, r, T, callPrice):
    return callPrice - S + K * exp(-r * T)

def ImpliedVolatility(Type, S, K, r, T, marketPrice):
    epsilon = 0.0000000001 # 可承受之誤差
    low = 0
    high = 2
    guess = (high + low) / 2
    numGuess = 0
    if Type == "Call":
        while abs(Black_Scholes_Call(S, K, r, T, guess) - marketPrice) >= epsilon:
            if Black_Scholes_Call(S, K, r, T, guess) < marketPrice:
                low = guess
            else:
                high = guess
            numGuess += 1
            guess = (high + low) / 2
    elif Type == "Put":
        while abs(European_Put(S, K, r, T, Black_Scholes_Call(S, K, r, T, guess)) - marketPrice) >= epsilon:
            if European_Put(S, K, r, T, Black_Scholes_Call(S, K, r, T, guess)) < marketPrice:
                low = guess
            else:
                high = guess
            numGuess += 1
            guess = (high + low) / 2
    print("Search times:{}".format(numGuess))
    return guess

In [3]:
ImpliedVolatility("Call", 9439, 9500, 0.01, 1/360, 34.981633711619)
# Note: 34.981633711619 is the theoratical price when volatility is 0.3042. Which confirms this code's validity.

Search times:39


0.30420000000049185

In [4]:
ImpliedVolatility("Put", 9439, 9500, 0.01, 1/360, 95.71774848781934)
# Note: 95.71774848781934 is the theoratical price when volatility is 0.3042. Which confirms this code's validity.

Search times:39


0.30420000000049185

In [5]:
# 若今S=9439, K=9500, r=0.01, T=1/360, 且Call市價為36, 則此option隱含波動度如下
ImpliedVolatility("Call", 9439, 9500, 0.01, 1/360, 36)

Search times:39


0.30973315388655465